In [ ]:
import numpy as np
import pandas as pd

health = pd.read_csv("../Datasets/rankmd.csv", delimiter=";")
FastFood = pd.read_csv("../Datasets/FastFoodRestaurants.csv")
income = pd.read_csv("../Datasets/kaggle_income.csv", encoding="ISO 8859-1")

us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

FastFood['State'] = FastFood['province'].map(abbrev_to_us_state)
States = health.copy()
States = States[States['FIPS'].astype(str).str.endswith('000')]
health['FIPS']=health['FIPS'].apply(lambda x: '{0:0>5}'.format(x))
health["food_environment_index_Food Environment Index"] = health["food_environment_index_Food Environment Index"].str.replace(",",".").astype(float)

In [ ]:
new_df = pd.merge(health.copy(), temp_df.copy(),  how='left', left_on=['State','County'], right_on = ['State_Name','County'])

In [ ]:
#ONLY NEEDS TO BE RAN ONCE AS THE COUNTIES ARE STORED IN THE CSV.

#!{sys.executable} -m pip install pgeocode
#import pgeocode

#nomi = pgeocode.Nominatim('us')
#county_names = []
#for i in range(len(FastFood)):
#    county_names.append(nomi.query_postal_code(FastFood["postalCode"][i]).county_name)
    
#FastFood["County"] = county_names
#FastFood.to_csv("../Datasets/FastFoodRestaurants.csv")

In [ ]:
temptemp = FastFood.groupby(["State", "County"]).count().reset_index()[['State','County','address']]
tempo = temptemp.rename(columns={'address':'nr of FFchains'})
data_df = pd.merge(new_df, tempo,  how='left', left_on=['State','County'], right_on =['State','County'])
data_df['nr of FFchains'] = data_df['nr of FFchains'].fillna(0)

data = data_df[["FIPS", "State", "County", "premature_deathYears_of_Potential_Life_Lost_Rate",'adult_obesity_% Adults with Obesity',
                 "adult_smoking_% Smokers", "excessive_drinking_% Excessive Drinking", "food_environment_index_Food Environment Index",
                 "uninsured_% Uninsured", "unemployed_% Unemployed", 'nr of FFchains', 'Mean']]
data = data.dropna()
data['is_obese'] = data['adult_obesity_% Adults with Obesity']>=33
data = data.drop(['adult_obesity_% Adults with Obesity'],axis=1)
data["unemployed_% Unemployed"] = data["unemployed_% Unemployed"].str.replace(",",".").astype(float).astype(int)

In [ ]:
data.to_csv("../Datasets/FastFoodRestaurants.csv")